# Problem Set #8
## Tc2006 Programming Languages

October 11, 2021.

_Authors of this notebook’s solution:_

- TEAM #
- Student ID and Name:
- Student ID and Name:

## Speedup

The speed of a program is the time it takes the program to execute. This could be measured in any increment of time. _Speedup_ is defined as the time it takes a program to execute sequentially (with one processor) divided by the time it takes to execute in parallel (with many processors). The formula for speedup is:

$$
S_p = \frac{T_1}{T_p}
$$

where:

* $S_p$ is the speedup obtained from using $p$ processors.
* $T_1$ is the time it takes the program to be executed sequentially.
* $T_p$ is the time it takes the program to be executed in parallel using $p$ processors.

_Linear speedup_ or _ideal speedup_ is obtained when:

$$
S_p = p
$$

When running an algorithm with linear speedup, doubling the number of processors doubles the speed. As this is ideal, it is considered very good scalability.

\[AKHTER\] pp. 13-15.

---
## Problem 1

Numerical integration is a method of computing an approximation of the area under the curve of a function, especially when the exact integral cannot be solved. For example, the value of the constant $\pi$ can be defined by the following integral:

$$
\pi = \int_{0}^{1}\frac{4}{1+x^2}\mathit{dx}
$$

However, rather than solve this integral exactly, we can approximate the solution by use of numerical integration. The following C code is an implementation of the numerical integration midpoint rectangle rule to solve the integral just shown. To compute an approximation of the area under the curve, we must compute the area of some number of rectangles (`num_rects`) by finding the midpoint (`mid`) of each rectangle and computing the height of that rectangle (`height`), which is simply the function value at that midpoint. We add together the heights of all the rectangles (`sum`) and, once computed, we multiply the sum of the heights by the width of the rectangles (`width`) to determine the desired approximation of the total area (`area`) and the value of $\pi$.

```c
double compute_pi(long num_rects) {
    double sum = 0.0;
    double width = 1.0 / (double) num_rects;
    for (long i = 0; i < num_rects; i++) {
        double mid = (i + 0.5) * width;
        double height = 4.0 / (1.0 + mid * mid);
        sum += height;
    }
    return width * sum;
}
```

Write an equivalent Clojure function `compute-pi`. Test the function with a large value of `num-rects` that takes several seconds to compute. In theory, you should get a better approximation of $\pi$ for larger values of `num-rects`. Compare your results with the first 20 decimal places of $\pi$: 

$$
3.14159265358979323846 \ldots
$$

Have in mind that 64-bit double precision floating point numbers are accurate up to sixteen decimal places but after calculations have been done there may be some rounding errors to account for.

Afterwards, modify your code so that it uses the `pmap` function to compute its result using a specified number of threads. Measure the time with 1 and $p$ threads, where $p$ is the number of logical CPU cores in your system. Calculate the speedup $S_p$. 

**TIPS:** Use the function (`time` $\textit{expr}$) to evaluate $\textit{expr}$ and print how many milliseconds it took. The function returns the result of $\textit{expr}$. Finally, you can get the number of logical cores with the following expression:

In [1]:
(.availableProcessors (Runtime/getRuntime))

4

In [2]:
(defn compute-pi
  [num-rects start end]
  (let [width (/ 1.0 num-rects)]
    (loop [sum 0.0
           i start]
      (if (< i end)
        (let [mid    (* (+ i 0.5) width)
              height (/ 4.0 (+ 1.0 (* mid mid)))]
          (recur (+ sum height) (inc i)))
        (* width sum)))))

#'user/compute-pi

In [3]:
(defn range-limits
  [num-rects num-threads]
  (partition 2 1 (range 0 (+ num-rects 1) (/ num-rects num-threads))))

#'user/range-limits

In [4]:
(defn pi
  [n p]
  (reduce +
          (pmap (fn [[start end]]
                  (compute-pi n start end))
                (range-limits n p))))

#'user/pi

In [5]:
(time (pi 1000000000 (.availableProcessors (Runtime/getRuntime))))

"Elapsed time: 3897.67132 msecs"


3.1415926535898215

$T_1 = 10.385s$

$T_4 = 3.661s$

$S_4 = 2.836$

In [6]:
(/ 10.385 3.661)

2.8366566511882

---
## Problem 2

If you convert the number 17 to binary and hexadecimal you get $10001_b$ and $11_h$, which happen to be palindromes (a sequence that reads the same backward as forward). Other numbers that have this same property are: 0 $(0_b, 0_h)$, 153 $(10011001_b, 99_h)$ and 255 $(11111111_b, \textrm{FF}_h)$. Let’s call these numbers _bin-hex-palindromes_.

Write a Clojure function `(count-bin-hex-palindromes n num-threads)` that counts the total number of _bin-hex-palindromes_ that are less than $2^n$. So, for example, if $n=7$ the function should return 11, because $2^7 = 128$, and there are 11 _bin-hex-palindromes_ that are less than 128: 

- _0_ $(0_b, 0_h)$
- _1_ $(1_b, 1_h)$
- _3_ $(11_b, 3_h)$
- _5_ $(101_b, 5_h)$ 
- _7_ $(111_b, 7_h)$
- _9_ $(1001_b, 9_h)$
- _15_ $(1111_b, \textrm{F}_h)$
- _17_ $(10001_b, 11_h)$
- _51_ $(110011_b, 33_h)$ 
- _85_ $(1010101_b, 55_h)$
- _119_ $(1110111_b, 77_h)$

The implementation must use `pmap` and consider the number of threads provided by `num-threads`, assuming that $n \ge \log_2(\texttt{num-threads})$.

Test your function with a value of $n$ that takes several seconds to compute. Measure the time with 1 and $p$ threads, where $p$ is the number of logical CPU cores in your system. Finally, compute the speedup $S_p$.

**TIPS:** The following Clojure and Java functions can be used to solve this problem:

In [ ]:
(Integer/toBinaryString 17)

In [ ]:
(Integer/toHexString 17)

In [ ]:
(reverse "abba") 

In [ ]:
(seq "abba")

In [ ]:
(= (seq "abba") (reverse "abba"))